<a href="https://colab.research.google.com/github/micheldocoder/ia-web-flask/blob/main/facialappcrypt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model.git


fatal: destination path 'first-order-model' already exists and is not an empty directory.


In [2]:
!pip install imageio imageio-ffmpeg matplotlib pyyaml ffmpeg-python
!pip install scikit-image==0.21.0 numpy==1.23.5


In [3]:
yaml_conteudo = """
model_params:
  common_params:
    num_kp: 10
    num_channels: 3
    estimate_jacobian: True

  kp_detector_params:
    temperature: 0.1
    block_expansion: 32
    max_features: 1024
    scale_factor: 0.25
    num_blocks: 5

  generator_params:
    block_expansion: 64
    max_features: 512
    num_down_blocks: 2
    num_bottleneck_blocks: 6
    estimate_occlusion_map: True
    dense_motion_params:
      block_expansion: 64
      max_features: 1024
      num_blocks: 5
      scale_factor: 0.25
"""

with open("first-order-model/config/vox-256.yaml", "w") as f:
    f.write(yaml_conteudo.strip())

print("✅ vox-256.yaml criado com sucesso!")


✅ vox-256.yaml criado com sucesso!


In [4]:
!mkdir -p first-order-model/checkpoints
!mv vox-cpk.pth first-order-model/checkpoints/vox-cpk.pth.tar
print("✅ Checkpoint movido e renomeado com sucesso!")


✅ Checkpoint movido e renomeado com sucesso!


In [6]:
import sys
sys.path.append('first-order-model')


In [7]:
from demo import load_checkpoints

config_path = 'first-order-model/config/vox-256.yaml'
checkpoint_path = 'first-order-model/checkpoints/vox-cpk.pth.tar'

generator, kp_detector = load_checkpoints(config_path, checkpoint_path, cpu=True)
print("✅ Modelo e kp_detector carregados com sucesso!")


/usr/local/lib/python3.11/dist-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


✅ Modelo e kp_detector carregados com sucesso!


In [10]:
import imageio.v2 as imageio
from skimage.transform import resize

# Caminhos corrigidos (sem sample_data)
source_image_path = 'f8b69eab-8af5-4a0c-b263-334d4464e9a4.jpg'
driving_video_path = 'WhatsApp Video 2025-03-31 at 10.17.26.mp4'

# Leitura da imagem
source_image = imageio.imread(source_image_path)
source_image = resize(source_image, (256, 256))[..., :3]

# Leitura do vídeo
reader = imageio.get_reader(driving_video_path)
fps = reader.get_meta_data()['fps']
driving_video = []

try:
    for frame in reader:
        driving_video.append(resize(frame, (256, 256))[..., :3])
except RuntimeError:
    print("⚠️ Problema ao ler algum frame do vídeo")
reader.close()

print("✅ Imagem e vídeo carregados com sucesso!")


✅ Imagem e vídeo carregados com sucesso!


In [11]:
from demo import make_animation
from skimage import img_as_ubyte
import imageio

# Geração da animação
predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True, adapt_movement_scale=True, cpu=True)

# Salvando o vídeo de saída
output_path = 'resultado.mp4'
imageio.mimsave(output_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)

print("✅ Animação gerada com sucesso e salva em 'resultado.mp4'")


  0%|          | 0/351 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/functional.py:5015: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


✅ Animação gerada com sucesso e salva em 'resultado.mp4'


In [12]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open('resultado.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f'<video width=400 controls><source src="{data_url}" type="video/mp4"></video>')
